In [126]:
# pip install python-dotenv

In [127]:
# !pip install openai

In [128]:
import os
from dotenv import load_dotenv
import openai
import json
import datetime
import sys

In [129]:
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

In [130]:
# import pandas as pd
# pd.get_option("display.max_columns")
# pd.get_option("display.max_colwidth")
# pd.set_option('display.max_columns', 50)
# pd.set_option('display.max_colwidth', 100)

In [131]:
# "get_belonging_occupation" function defined as Function Calling
# e.g., industry_answer: [{"job title": "Software Engineer", "industry name": "Technology"}, {"job title": "Data Scientist", "industry name": "Science"}, {"job title": "Front-end engineer", "industry name": "To be predicted"}]

def get_belonging_occupation(jobs):
    d = {"Software Engineer": "Technology", "Data Scientist": "Science"}

    # Receives a list of job titles and returns a dict with the corresponding information for each and the industry. However, if there is no specific definition, "To be predicted" is stored.
    def search_industry(job):
        # print(job)
        # print(d.get(job, "To be predicted"))
        return d.get(job, "To be predicted")

    industry_answer = [
        {"job title": job.strip(), "industry name": search_industry(job.strip())} for job in jobs.split(",")
    ]
    return json.dumps(industry_answer)

- 1st step : Provide definition of functions to be used by GPT

In [132]:
# 1st step : Provide definition of "get_belonging_occupation" function to be used by GPT

functions = [
    # Whether GPT uses this function for questions,
    # Also give information to determine what arguments to use when using it.
    {
        "name": "get_belonging_occupation",
        "description": "Get the name of the industry that corresponds to the given job title.",
        "parameters": {
            "type": "object",
            "properties": {
                # Information about the jobs argument
                "jobs": {
                    "type": "string",
                    "description": "Enter job title. Multiple elements can be entered separated by single-byte commas. e.g., Software Engineer, Data Scientist",
                },
            },
            "required": ["jobs"],
        },
    }
]

In [133]:
# question = "What occupation does the following job title belong to? Software Engineer,Data Scientist"
# question = "What is the common point of the following job titles? Software Engineer,Data Scientist"
question = "Are the industry of Software Engineer, Data Scientist and Front-end engineer the same?"
# question = "Please provide a list of industry."

- 2nd step : Pass the function definition and question to GPT to determine which function to call.

In [134]:
# 2nd step : Pass the function definition and question to GPT to determine which function to call.

response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        {"role": "user", "content": question},
    ],
    functions=functions,
    function_call="auto",
)
print(json.dumps(response), file=sys.stderr)

{"id": "chatcmpl-8jhq0KLpQwpvjlM9Qp5plULyBQl4m", "object": "chat.completion", "created": 1705903964, "model": "gpt-4-0613", "choices": [{"index": 0, "message": {"role": "assistant", "content": null, "function_call": {"name": "get_belonging_occupation", "arguments": "{\n  \"jobs\": \"Software Engineer, Data Scientist, Front-end engineer\"\n}"}}, "logprobs": null, "finish_reason": "function_call"}], "usage": {"prompt_tokens": 98, "completion_tokens": 27, "total_tokens": 125}, "system_fingerprint": null}


- 3rd step : Call the function determined to be used by GPT and get the result
- 4th step : Pass the function response to GPT and get the final response

In [135]:
message = response["choices"][0]["message"]

# If it is determined that the function
if message.get("function_call"):

    # Function name determined to be used
    function_name = message["function_call"]["name"]
    
    # Argument dict determined to be used
    arguments = json.loads(message["function_call"]["arguments"])

    # 3rd step : Call the function determined to be used by GPT and get the result
    function_response = eval(function_name)(jobs=arguments.get("jobs")) # function_name: get_belonging_occupation, arguments: {'jobs': 'Software Engineer, Data Scientist, Front-end engineer'}
    # function_response = get_belonging_occupation(
    #     jobs=arguments.get("jobs"),
    # )

    print("function_response: " + function_response, file=sys.stderr)

    # 4th step : Pass the function response to GPT and get the final response
    second_response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[
            {"role": "user", "content": question},
            message,
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            },
        ],
    )

    print(json.dumps(second_response), file=sys.stderr)
    print(second_response.choices[0]["message"]["content"].strip())

function_response: [{"job title": "Software Engineer", "industry name": "Technology"}, {"job title": "Data Scientist", "industry name": "Science"}, {"job title": "Front-end engineer", "industry name": "To be predicted"}]


The primary industry for both Software Engineers, Data Scientists, and Front-end Engineers is Technology. However, Data Scientists also significantly intersect with the Science industry, especially in fields like data analysis and machine learning.


{"id": "chatcmpl-8jhq23hrnOkSuWxjH9cw4TqG26uGs", "object": "chat.completion", "created": 1705903966, "model": "gpt-4-0613", "choices": [{"index": 0, "message": {"role": "assistant", "content": "The primary industry for both Software Engineers, Data Scientists, and Front-end Engineers is Technology. However, Data Scientists also significantly intersect with the Science industry, especially in fields like data analysis and machine learning."}, "logprobs": null, "finish_reason": "stop"}], "usage": {"prompt_tokens": 112, "completion_tokens": 40, "total_tokens": 152}, "system_fingerprint": null}
